In [5]:
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output
import requests
import math
import keyboard

base_url = 'https://www.cheapshark.com/api/1.0/games'


search_button = widgets.Button(description='Search')
title_input = widgets.Text(value='', description='Enter Game:')

search_box = widgets.HBox([title_input, search_button])
search_box.layout.width = '400px'
output = widgets.Output()  # Use the same output widget consistently
page_number = widgets.IntText(value=1, description='Page:')
prev_page_button = widgets.Button(description='Previous Page', disabled=True)
next_page_button = widgets.Button(description='Next Page')
page_size = 10  # Number of results per page

options = [
    ("Price Ascending", "asc"),
    ("Price Descending", "desc"),
    ("A-Z", "az")
]
dropdown = widgets.Dropdown(
    options=options,
    description='Sort by:',
    value="asc"
)

sorting_type = dropdown.value

searched = False  # Track whether a search has been performed

def search_game(b):
    global searched
    global total_pages
    searched = True
    prev_page_button.disabled = True
    with output:
        clear_output()
        title = title_input.value
        results, total_results = get_game_results(title)
        
        if results:
            total_pages = math.ceil(y/page_size)
            # Reset the page number to 1 when a new search is performed
            page_number.value /= page_number.value
            display_pagination()
            display_paginated_results(results, page_number.value, page_size)
            display(widgets.HBox([prev_page_button, next_page_button]))
            

def get_game_results(title):
    global results, y
    url = f'{base_url}?title={title}'
    response = requests.get(url).json()
    results = []
    x=[]

    for game in response:
        external = game['external']
        image_url = game['thumb']
        deal_id = game['cheapestDealID']
        price = game['cheapest']
        results.append((external, image_url, deal_id, price))
        x.append(external)
    y = len(x)
    return results, y


                     
def on_dropdown_change(change):
    selected_option = change['new']
#     print(selected_option)
    sort_and_display_results(selected_option, results)     
    

def sort_and_display_results(change, results):
    global sorted_results
    selected_option = change
#     print(change)
    page_number.value /= page_number.value
    with output:
        if selected_option == "asc":
            sorted_results = sorted(results, key=lambda x: float(x[3]))
        elif selected_option == "desc":
            sorted_results = sorted(results, key=lambda x: float(x[3]), reverse=True)
        elif selected_option == "az":
            sorted_results = sorted(results, key=lambda x: x[0])
           
    with output:
        clear_output()
        title = title_input.value
        results, total_results = get_game_results(title)
        if results:
            display_pagination()
            display_paginated_results(sorted_results, page_number.value, page_size)
            display(widgets.HBox([prev_page_button, next_page_button]))




def display_pagination():
    if searched:
        display(Markdown(f"Page: {page_number.value}"))
        display(dropdown)
        
def display_paginated_results(results, page, page_size):
    
    start_index = (page - 1 ) * page_size
    end_index = start_index + page_size
    
    for result in results[start_index:end_index]:
        title, image_url, deal_id, price = result
        display(Markdown(f"## {title}"))
        display(HTML(f'<a href="https://www.cheapshark.com/redirect?dealID={deal_id}" target="_blank"><img src="{image_url}" alt="{title}" width="200"></a>'))
        print(f"Price: ${price}")

def previous_page(b):
    if page_number.value > 1:
        clear_output()
        page_number.value -= 1
        update_pagination_buttons()
        display_pagination()
        display_paginated_results(sorted_results, page_number.value, page_size)
        display(widgets.HBox([prev_page_button, next_page_button]))
    if page_number.value == 1:
        prev_page_button.diasbled = True

def next_page(b):
    if page_number.value < total_pages:
        clear_output()
        page_number.value += 1
        update_pagination_buttons()
        display_pagination()
        display_paginated_results(sorted_results, page_number.value, page_size)
        display(widgets.HBox([prev_page_button, next_page_button]))
    if page_number.value == total_pages:
        next_page_button.diasbled = True

def update_pagination_buttons():
    print(page_number.value)
    if page_number.value >= total_pages:
        next_page_button.disabled = True
        page_number.value = total_pages
    else:
        next_page_button.disabled = False
    
    if page_number.value <= 1:
        prev_page_button.disabled = True
        page_number.value = 1
    else:
        prev_page_button.disabled = False
    
def on_key_event(e):
    if e.event_type == keyboard.KEY_DOWN:
        if e.name == 'enter':
            search_button.click()  
        elif e.name == 'right':
            next_page_button.click() 
        elif e.name == 'left':
            prev_page_button.click()
    
        
# Button click events
keyboard.hook(on_key_event)
dropdown.observe(on_dropdown_change,  names='value')
search_button.on_click(search_game)
prev_page_button.on_click(previous_page)
next_page_button.on_click(next_page)

# Display widgets
display(search_box, output)



6


Page: 6

Dropdown(description='Sort by:', options=(('Price Ascending', 'asc'), ('Price Descending', 'desc'), ('A-Z', 'a…

## Slab

Price: $1.99


...

Price: $0.99


## Slip

Price: $0.99


## Gelu

Price: $0.49
